<a href="https://colab.research.google.com/github/carloscesar182/ai_advanced_course/blob/main/Notebooks/RNA/DeteccaoObjetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
# criar 2 variaveis: caminho do modelo e caminho dos pesos
MODEL_CONFIG = "MobileNetSSD_deploy.prototxt"
MODEL_WEIGHTS = "MobileNetSSD_deploy.caffemodel"

# criar uma lista com as classes
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat", "bottle", "bus",
          "car", "cat", "chair", "cow", "diningtable", "dog", "horse", "motorbike",
           "person", "pottedplant", "sheep", "sofa", "train", "tvmonitor"]

In [ ]:
# carregar o modelo
net = cv2.dnn.readNetFromCaffe(MODEL_CONFIG, MODEL_WEIGHTS)

In [ ]:
# criar uma função que processa a detecção do objeto
# ela vai receber o caminho do video, carregar o video, rodar o video até acabar ou detectar o objeto
def detect_person_in_video(video_path):
  cap = cv2.VideoCapture(video_path) # carrega o video
  frame_count = 0 # variável de controle pra contar os frames
  while True:
    ret, frame = cap.read()
    if not ret:
      print("Fim do video ou erro de leitura")
      break
    frame_count += 1

    # pegar largura e altura do objeto
    (h, w) = frame.shape[:2]

    # preprocessar o frame
    # frame + boas práticas de normalização + tamanho + valor a ser subtraído
    blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

    # processar na rede neural
    net.setInput(blob)

    # o objeto detection é uma lista com 4 valores
    # batch 0, indice = id da classe, confiança, coordenadas
    detection = net.forward() # faz a deteção de objetos
    for i in range(detection.shape[2]): # na posição 2 estão as detecções
      confidence = detection[0, 0, i, 2] # pega a confiança da detecção
      if confidence > 0.9: # se a confiança for maior que 0.9 (bem alta)
        idx = int(detection[0, 0, i, 1])
        if CLASSES[idx] == "person": # se for pessoa
          # pegar as coordenadas da caixa onde detectou o objeto
          box = detection[0, 0, i, 3:7] * np.array([w, h, w, h])
          (startX, startY, endX, endY) = box.astype("int")
          # desenhar a caixa com label e retangulo
          label = f"{CLASSES[idx]}: {confidence * 100:.2f}%"
          cv2.rectangle(frame, (startX, startY), (endX, endY), (0, 255, 0), 2)
          # desenhar o texto
          cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
          print(f"Pessoa detectada no frame {frame_count}")
          # mostrar o frame
          cv2_imshow(frame)
          detected_frame_path = f"detected_frame_{frame_count}.jpg"
          cv2.imwrite(detected_frame_path, frame)
          print(f"Frame salvo em: {detected_frame_path}")

          # liberar o objeto
          cap.release()
          return "Pessoa detectada no video"
  cap.release()
  return "Nenhuma pessoa detectada no video"

In [ ]:
print(detect_person_in_video("nopeople.mp4"))